# SETUP

## is GPU on?

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## boilerplate

In [1]:
# %%capture cap --no-stderr
from tensorflow.keras.callbacks import Callback, EarlyStopping
from numpy import arange

MODEL_NAME = 'mobilenet'
IMG_SIZE = (224, 224)
INPUT_SHAPE=(224, 224, 3)
CLASSES = 2
FT_BLOCK = 10 # FROM feature extractor TO fine tuning scratch
BATCH_SIZE = 64 # these data points will be passed as a batch at one time to the network
MODEL_DIR = '../../models/PR'

EPOCH = 100000
pr=9
tag = f'{MODEL_NAME}_BS{BATCH_SIZE}_FT{FT_BLOCK*10}_PR{pr*10}_epoch{EPOCH}'
print(tag)

mobilenet_BS64_FT100_PR90_epoch100000


## UDC/Fs

In [2]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras import layers
from math import floor
from timeit import default_timer as timer
from json import dump
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import data
from keras.engine import training

def model_constructor(FT_BLOCK):
    base_model = MobileNet(
        weights='imagenet', 
        include_top=False,
        input_shape=INPUT_SHAPE)
    
    # construct the top layer containing 
    x = base_model.output
    x = layers.GlobalAveragePooling2D(keepdims=True)(x)
    x = layers.Dropout(1e-3, name='dropout')(x)
    x = layers.Conv2D(CLASSES, (1, 1), padding='same', name='conv_preds')(x)
    x = layers.Reshape((CLASSES,), name='reshape_2')(x)
    x = layers.Activation(activation='sigmoid', name='predictions')(x)
    
    # model is ready to be trained
    alpha, rows = 1.0, 224
    model = training.Model(inputs=base_model.input, outputs=x, name='mobilenet_%0.2f_%s' % (alpha, rows))

    # freeze the layers before the `FROZEN_END` layer 
    total_layers = len(base_model.layers)
    ft_layers = floor(total_layers / 10)
    FROZEN_END = total_layers - ft_layers * FT_BLOCK

    base_model.trainable = True
    for layer in base_model.layers[:FROZEN_END]: 
        layer.trainable = False
    
    return model

class TimeCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def time_converter(sec):
    hours, rem = divmod(sec, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        
def data_preparation(BATCH_SIZE):
    train_dir = '../../data/binaryclass_clean/train/'
    test_dir = '../../data/binaryclass_clean/test/'
    
    train_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='training')

    val_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='validation')

    test_ds = image_dataset_from_directory(
        directory=test_dir,
        label_mode='categorical',
        batch_size=1,
        image_size=IMG_SIZE)

    AUTOTUNE = data.AUTOTUNE
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

import tempfile
from os.path import getsize

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in Mbytes.
  import os
  import zipfile

  zipped_file = f'{MODEL_DIR}/{tag}.zip'
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
    zipped_size = getsize(zipped_file) / float(2**20)
#     remove(zipped_file)

  return zipped_size

In [3]:
train_ds, val_ds, test_ds = data_preparation(BATCH_SIZE)

Found 9980 files belonging to 2 classes.
Using 8982 files for training.
Found 9980 files belonging to 2 classes.
Using 998 files for validation.
Found 1000 files belonging to 2 classes.


# EPOCH 100K

In [ ]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import save_model

logdir = f'../../logs/PR/{tag}.json'

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(
        target_sparsity=pr/10, begin_step=0, end_step=-1, frequency=1)}

model = model_constructor(FT_BLOCK)
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(model, **pruning_params)
# model_for_pruning.summary()

model_for_pruning.compile(loss='binary_crossentropy', metrics='accuracy', optimizer = 'adam')
history = model_for_pruning.fit(train_ds, epochs=EPOCH, verbose=1, validation_data=val_ds, callbacks=callbacks)

loss, accuracy = model_for_pruning.evaluate(test_ds)

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
_, pruned_keras_file = tempfile.mkstemp('.h5')
save_model(model_for_export, pruned_keras_file)
model_size = get_gzipped_model_size(pruned_keras_file)

/usr/local/lib/python3.6/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/100000
141/141 [==============================] - 60s 261ms/step - loss: 0.5450 - accuracy: 0.7229 - val_loss: 1.2181 - val_accuracy: 0.4419
Epoch 2/100000
141/141 [==============================] - 35s 236ms/step - loss: 0.4121 - accuracy: 0.8057 - val_loss: 0.6882 - val_accuracy: 0.5741
Epoch 3/100000
141/141 [==============================] - 34s 234ms/step - loss: 0.3463 - accuracy: 0.8400 - val_loss: 0.7327 - val_accuracy: 0.5631
Epoch 4/100000
141/141 [==============================] - 33s 227ms/step - loss: 0.2733 - accuracy: 0.8798 - val_loss: 0.6167 - val_accuracy: 0.6393
Epoch 5/100000
141/141 [==============================] - 34s 233ms/step - loss: 0.2340 - accuracy: 0.8969 - val_loss: 0.7201 - val_accuracy: 0.5561
Epoch 6/100000
141/141 [==============================] - 35s 240ms/step - loss: 0.1877 - accuracy: 0.9179 - val_loss: 0.6654 - val_accuracy: 0.5782
Epoch 7/100000
141/141 [==============================] - 34s 231ms/step - loss: 0.1469 - accuracy: 0.9351

Epoch 54/100000
141/141 [==============================] - 34s 230ms/step - loss: 0.0223 - accuracy: 0.9908 - val_loss: 1.0960 - val_accuracy: 0.7335
Epoch 55/100000
141/141 [==============================] - 34s 229ms/step - loss: 0.0351 - accuracy: 0.9856 - val_loss: 0.8359 - val_accuracy: 0.7535
Epoch 56/100000
141/141 [==============================] - 34s 230ms/step - loss: 0.0397 - accuracy: 0.9831 - val_loss: 0.8728 - val_accuracy: 0.6964
Epoch 57/100000
141/141 [==============================] - 34s 236ms/step - loss: 0.0255 - accuracy: 0.9883 - val_loss: 1.2161 - val_accuracy: 0.7315
Epoch 58/100000
141/141 [==============================] - 34s 230ms/step - loss: 0.0217 - accuracy: 0.9903 - val_loss: 1.6247 - val_accuracy: 0.6433
Epoch 59/100000
141/141 [==============================] - 34s 233ms/step - loss: 0.0220 - accuracy: 0.9906 - val_loss: 2.0487 - val_accuracy: 0.7425
Epoch 60/100000
141/141 [==============================] - 34s 230ms/step - loss: 0.0206 - accuracy:

141/141 [==============================] - 35s 237ms/step - loss: 0.0115 - accuracy: 0.9939 - val_loss: 1.0502 - val_accuracy: 0.7435
Epoch 163/100000
141/141 [==============================] - 33s 229ms/step - loss: 0.0124 - accuracy: 0.9943 - val_loss: 1.3212 - val_accuracy: 0.7936
Epoch 164/100000
141/141 [==============================] - 34s 234ms/step - loss: 0.0163 - accuracy: 0.9916 - val_loss: 1.5777 - val_accuracy: 0.7585
Epoch 165/100000
141/141 [==============================] - 34s 235ms/step - loss: 0.0128 - accuracy: 0.9934 - val_loss: 1.4343 - val_accuracy: 0.8126
Epoch 166/100000
141/141 [==============================] - 34s 231ms/step - loss: 0.0121 - accuracy: 0.9939 - val_loss: 2.1479 - val_accuracy: 0.7796
Epoch 167/100000
141/141 [==============================] - 34s 234ms/step - loss: 0.0116 - accuracy: 0.9939 - val_loss: 2.0959 - val_accuracy: 0.7114
Epoch 168/100000
141/141 [==============================] - 33s 229ms/step - loss: 0.0088 - accuracy: 0.9951 - 

141/141 [==============================] - 34s 232ms/step - loss: 0.0075 - accuracy: 0.9959 - val_loss: 1.1669 - val_accuracy: 0.8267
Epoch 271/100000
141/141 [==============================] - 34s 231ms/step - loss: 0.0081 - accuracy: 0.9954 - val_loss: 2.5598 - val_accuracy: 0.7675
Epoch 272/100000
141/141 [==============================] - 33s 228ms/step - loss: 0.0095 - accuracy: 0.9945 - val_loss: 2.7389 - val_accuracy: 0.6393
Epoch 273/100000
141/141 [==============================] - 34s 233ms/step - loss: 0.0197 - accuracy: 0.9915 - val_loss: 3.4038 - val_accuracy: 0.5982
Epoch 274/100000
141/141 [==============================] - 34s 233ms/step - loss: 0.0150 - accuracy: 0.9935 - val_loss: 1.4327 - val_accuracy: 0.7715
Epoch 275/100000
141/141 [==============================] - 34s 233ms/step - loss: 0.0144 - accuracy: 0.9924 - val_loss: 1.5485 - val_accuracy: 0.7906
Epoch 276/100000
141/141 [==============================] - 34s 230ms/step - loss: 0.0103 - accuracy: 0.9947 - 

...

141/141 [==============================] - 31s 211ms/step - loss: 0.0027 - accuracy: 0.9987 - val_loss: 2.5919 - val_accuracy: 0.8397
Epoch 20009/100000
141/141 [==============================] - 31s 213ms/step - loss: 0.0027 - accuracy: 0.9986 - val_loss: 2.5966 - val_accuracy: 0.8387
Epoch 20010/100000
141/141 [==============================] - 31s 211ms/step - loss: 0.0022 - accuracy: 0.9988 - val_loss: 2.6026 - val_accuracy: 0.8427
Epoch 20011/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0037 - accuracy: 0.9983 - val_loss: 2.5567 - val_accuracy: 0.8457
Epoch 20012/100000
141/141 [==============================] - 31s 211ms/step - loss: 0.0028 - accuracy: 0.9986 - val_loss: 2.5562 - val_accuracy: 0.8447
Epoch 20013/100000
141/141 [==============================] - 31s 210ms/step - loss: 0.0026 - accuracy: 0.9988 - val_loss: 2.5930 - val_accuracy: 0.8427
Epoch 20014/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0033 - accurac

141/141 [==============================] - 31s 212ms/step - loss: 0.0025 - accuracy: 0.9989 - val_loss: 2.6968 - val_accuracy: 0.8437
Epoch 20115/100000
141/141 [==============================] - 31s 211ms/step - loss: 0.0028 - accuracy: 0.9987 - val_loss: 2.6976 - val_accuracy: 0.8367
Epoch 20116/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0029 - accuracy: 0.9984 - val_loss: 2.6818 - val_accuracy: 0.8427
Epoch 20117/100000
141/141 [==============================] - 31s 211ms/step - loss: 0.0028 - accuracy: 0.9988 - val_loss: 2.6116 - val_accuracy: 0.8407
Epoch 20118/100000
141/141 [==============================] - 31s 211ms/step - loss: 0.0028 - accuracy: 0.9986 - val_loss: 2.6405 - val_accuracy: 0.8317
Epoch 20119/100000
141/141 [==============================] - 31s 213ms/step - loss: 0.0028 - accuracy: 0.9987 - val_loss: 2.6810 - val_accuracy: 0.8307
Epoch 20120/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0026 - accurac

141/141 [==============================] - 31s 214ms/step - loss: 0.0029 - accuracy: 0.9988 - val_loss: 2.6156 - val_accuracy: 0.8347
Epoch 20221/100000
141/141 [==============================] - 31s 214ms/step - loss: 0.0026 - accuracy: 0.9987 - val_loss: 2.7139 - val_accuracy: 0.8357
Epoch 20222/100000
141/141 [==============================] - 31s 211ms/step - loss: 0.0030 - accuracy: 0.9984 - val_loss: 2.6616 - val_accuracy: 0.8437
Epoch 20223/100000
141/141 [==============================] - 31s 210ms/step - loss: 0.0025 - accuracy: 0.9989 - val_loss: 2.6098 - val_accuracy: 0.8457
Epoch 20224/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0031 - accuracy: 0.9984 - val_loss: 2.6269 - val_accuracy: 0.8427
Epoch 20225/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0029 - accuracy: 0.9986 - val_loss: 2.5515 - val_accuracy: 0.8447
Epoch 20226/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0024 - accurac

141/141 [==============================] - 31s 212ms/step - loss: 0.0033 - accuracy: 0.9981 - val_loss: 2.5252 - val_accuracy: 0.8497
Epoch 20327/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0026 - accuracy: 0.9986 - val_loss: 2.6195 - val_accuracy: 0.8427
Epoch 20328/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0028 - accuracy: 0.9986 - val_loss: 2.5950 - val_accuracy: 0.8427
Epoch 20329/100000
141/141 [==============================] - 31s 213ms/step - loss: 0.0026 - accuracy: 0.9987 - val_loss: 2.7457 - val_accuracy: 0.8447
Epoch 20330/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0034 - accuracy: 0.9986 - val_loss: 2.7638 - val_accuracy: 0.8457
Epoch 20331/100000
141/141 [==============================] - 31s 214ms/step - loss: 0.0029 - accuracy: 0.9986 - val_loss: 2.5564 - val_accuracy: 0.8457
Epoch 20332/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0041 - accurac

141/141 [==============================] - 31s 211ms/step - loss: 0.0027 - accuracy: 0.9988 - val_loss: 2.5324 - val_accuracy: 0.8487
Epoch 20433/100000
141/141 [==============================] - 31s 211ms/step - loss: 0.0026 - accuracy: 0.9987 - val_loss: 2.5423 - val_accuracy: 0.8457
Epoch 20434/100000
141/141 [==============================] - 31s 210ms/step - loss: 0.0025 - accuracy: 0.9989 - val_loss: 2.5917 - val_accuracy: 0.8407
Epoch 20435/100000
141/141 [==============================] - 31s 210ms/step - loss: 0.0034 - accuracy: 0.9986 - val_loss: 3.1532 - val_accuracy: 0.8267
Epoch 20436/100000
141/141 [==============================] - 31s 211ms/step - loss: 0.0026 - accuracy: 0.9989 - val_loss: 2.8659 - val_accuracy: 0.8397
Epoch 20437/100000
141/141 [==============================] - 31s 212ms/step - loss: 0.0031 - accuracy: 0.9986 - val_loss: 2.8459 - val_accuracy: 0.8387
Epoch 20438/100000
141/141 [==============================] - 31s 213ms/step - loss: 0.0027 - accurac

In [6]:
print(loss, accuracy, model_size)

5.0028791427612305 0.7860000133514404 2.6955137252807617
